In [1]:
import pickle
import pandas as pd
import numpy as np
import requests
import calc

In [2]:
filename = 'btcusdr_lightgbm_model.pkl'
# 保存したモデルの読み込み
model = pickle.load(open(filename, 'rb'))

In [24]:
rsp = requests.get('https://api.bybit.com/v5/market/kline?category=linear&symbol=BTCUSDT&interval=1').json()
results = rsp['result']
df = pd.DataFrame(results['list'],columns=['startTime','Open','High','Low','Close','Volume','turnover'])

In [25]:
df["startTime"]=df["startTime"].astype(float)
df["Open"]=df["Open"].astype(float)
df["High"]=df["High"].astype(float)
df["Low"]=df["Low"].astype(float)
df["Close"]=df["Close"].astype(float)
df["Volume"]=df["Volume"].astype(float)

In [26]:
# 先頭行を削除
df = df.drop(df.index[0])
df.reset_index(inplace=True, drop=True)

# 時間でソート
df = df.sort_values('startTime')

# Timestamp列を追加
df['Timestamp'] = pd.to_datetime(df['startTime'], unit='ms')


In [27]:
# 短期および長期のSMAを計算して新たなカラムとして追加
short_term_window = 5  # 短期SMAのウィンドウ期間（例：5分）
long_term_window = 20  # 長期SMAのウィンドウ期間（例：20分）

df['SMA_short_term'] = calc.calculate_sma(df, 'Close', short_term_window)
df['SMA_long_term'] = calc.calculate_sma(df, 'Close', long_term_window)

# 結果を表示
print(df.tail())

      startTime     Open     High      Low    Close    Volume       turnover  \
4  1.699547e+12  36530.1  36621.3  36481.0  36545.0   995.140   36378978.228   
3  1.699547e+12  36545.0  36737.6  36506.0  36703.8   717.369  26289545.0936   
2  1.699547e+12  36703.8  36728.8  36620.0  36625.8   608.348  22319278.1389   
1  1.699547e+12  36625.8  36661.7  36571.6  36587.9   975.351  35709916.6563   
0  1.699547e+12  36587.9  36735.9  36587.9  36688.1  1035.130  37977385.1119   

            Timestamp  SMA_short_term  SMA_long_term  
4 2023-11-09 16:17:00        36573.34      36894.500  
3 2023-11-09 16:18:00        36619.58      36873.190  
2 2023-11-09 16:19:00        36621.12      36847.320  
1 2023-11-09 16:20:00        36598.52      36816.090  
0 2023-11-09 16:21:00        36630.12      36788.895  


In [28]:
# ボリンジャーバンドの上限、下限、中心線を計算して新たなカラムとして追加
window = 20  # ボリンジャーバンドのウィンドウ期間（例：20分）
num_std_dev = 2  # 標準偏差の倍数（通常は2）

df['BB_upper'], df['BB_lower'], df['BB_middle'] = calc.calculate_bollinger_bands(df, 'Close', window, num_std_dev)

# 結果を表示
print(df.tail())

      startTime     Open     High      Low    Close    Volume       turnover  \
4  1.699547e+12  36530.1  36621.3  36481.0  36545.0   995.140   36378978.228   
3  1.699547e+12  36545.0  36737.6  36506.0  36703.8   717.369  26289545.0936   
2  1.699547e+12  36703.8  36728.8  36620.0  36625.8   608.348  22319278.1389   
1  1.699547e+12  36625.8  36661.7  36571.6  36587.9   975.351  35709916.6563   
0  1.699547e+12  36587.9  36735.9  36587.9  36688.1  1035.130  37977385.1119   

            Timestamp  SMA_short_term  SMA_long_term      BB_upper  \
4 2023-11-09 16:17:00        36573.34      36894.500  37482.264403   
3 2023-11-09 16:18:00        36619.58      36873.190  37455.886479   
2 2023-11-09 16:19:00        36621.12      36847.320  37425.466478   
1 2023-11-09 16:20:00        36598.52      36816.090  37378.442101   
0 2023-11-09 16:21:00        36630.12      36788.895  37318.194121   

       BB_lower  BB_middle  
4  36306.735597  36894.500  
3  36290.493521  36873.190  
2  36269.17

In [29]:
df.dtypes

startTime                float64
Open                     float64
High                     float64
Low                      float64
Close                    float64
Volume                   float64
turnover                  object
Timestamp         datetime64[ns]
SMA_short_term           float64
SMA_long_term            float64
BB_upper                 float64
BB_lower                 float64
BB_middle                float64
dtype: object

In [30]:
# RSIを計算して新たなカラムとして追加
window = 14  # RSIのウィンドウ期間（例：14分）

df['RSI'] = calc.calculate_rsi(df, 'Close', window)

# 結果を表示
print(df.tail())

      startTime     Open     High      Low    Close    Volume       turnover  \
4  1.699547e+12  36530.1  36621.3  36481.0  36545.0   995.140   36378978.228   
3  1.699547e+12  36545.0  36737.6  36506.0  36703.8   717.369  26289545.0936   
2  1.699547e+12  36703.8  36728.8  36620.0  36625.8   608.348  22319278.1389   
1  1.699547e+12  36625.8  36661.7  36571.6  36587.9   975.351  35709916.6563   
0  1.699547e+12  36587.9  36735.9  36587.9  36688.1  1035.130  37977385.1119   

            Timestamp  SMA_short_term  SMA_long_term      BB_upper  \
4 2023-11-09 16:17:00        36573.34      36894.500  37482.264403   
3 2023-11-09 16:18:00        36619.58      36873.190  37455.886479   
2 2023-11-09 16:19:00        36621.12      36847.320  37425.466478   
1 2023-11-09 16:20:00        36598.52      36816.090  37378.442101   
0 2023-11-09 16:21:00        36630.12      36788.895  37318.194121   

       BB_lower  BB_middle        RSI  
4  36306.735597  36894.500  29.532924  
3  36290.493521  3

In [31]:
# MACDおよびシグナルラインを計算して新たなカラムとして追加
short_window = 12  # 短期EMAのウィンドウ期間（例：12分）
long_window = 26   # 長期EMAのウィンドウ期間（例：26分）
signal_window = 9  # シグナルラインのウィンドウ期間（例：9分）

df['MACD'], df['Signal_line'] = calc.calculate_macd(df, 'Close', short_window, long_window, signal_window)

# 結果を表示
print(df.tail())

      startTime     Open     High      Low    Close    Volume       turnover  \
4  1.699547e+12  36530.1  36621.3  36481.0  36545.0   995.140   36378978.228   
3  1.699547e+12  36545.0  36737.6  36506.0  36703.8   717.369  26289545.0936   
2  1.699547e+12  36703.8  36728.8  36620.0  36625.8   608.348  22319278.1389   
1  1.699547e+12  36625.8  36661.7  36571.6  36587.9   975.351  35709916.6563   
0  1.699547e+12  36587.9  36735.9  36587.9  36688.1  1035.130  37977385.1119   

            Timestamp  SMA_short_term  SMA_long_term      BB_upper  \
4 2023-11-09 16:17:00        36573.34      36894.500  37482.264403   
3 2023-11-09 16:18:00        36619.58      36873.190  37455.886479   
2 2023-11-09 16:19:00        36621.12      36847.320  37425.466478   
1 2023-11-09 16:20:00        36598.52      36816.090  37378.442101   
0 2023-11-09 16:21:00        36630.12      36788.895  37318.194121   

       BB_lower  BB_middle        RSI        MACD  Signal_line  
4  36306.735597  36894.500  29.53

In [32]:
# ヒストリカルボラティリティを計算して新たなカラムとして追加
window = 14  # ヒストリカルボラティリティのウィンドウ期間（例：14分）

df['Historical_volatility'] = calc.calculate_historical_volatility(df, 'Close', window)

# 結果を表示
print(df.tail())

      startTime     Open     High      Low    Close    Volume       turnover  \
4  1.699547e+12  36530.1  36621.3  36481.0  36545.0   995.140   36378978.228   
3  1.699547e+12  36545.0  36737.6  36506.0  36703.8   717.369  26289545.0936   
2  1.699547e+12  36703.8  36728.8  36620.0  36625.8   608.348  22319278.1389   
1  1.699547e+12  36625.8  36661.7  36571.6  36587.9   975.351  35709916.6563   
0  1.699547e+12  36587.9  36735.9  36587.9  36688.1  1035.130  37977385.1119   

            Timestamp  SMA_short_term  SMA_long_term      BB_upper  \
4 2023-11-09 16:17:00        36573.34      36894.500  37482.264403   
3 2023-11-09 16:18:00        36619.58      36873.190  37455.886479   
2 2023-11-09 16:19:00        36621.12      36847.320  37425.466478   
1 2023-11-09 16:20:00        36598.52      36816.090  37378.442101   
0 2023-11-09 16:21:00        36630.12      36788.895  37318.194121   

       BB_lower  BB_middle        RSI        MACD  Signal_line  \
4  36306.735597  36894.500  29.5

In [33]:
# ストキャスティクスを計算して新たなカラムとして追加
k_window = 14  # %Kのウィンドウ期間（例：14分）
d_window = 3   # %Dのウィンドウ期間（例：3分）

df['%K'], df['%D'] = calc.calculate_stochastics(df, 'High', 'Low', 'Close', k_window, d_window)

# 結果を表示
print(df.tail())

      startTime     Open     High      Low    Close    Volume       turnover  \
4  1.699547e+12  36530.1  36621.3  36481.0  36545.0   995.140   36378978.228   
3  1.699547e+12  36545.0  36737.6  36506.0  36703.8   717.369  26289545.0936   
2  1.699547e+12  36703.8  36728.8  36620.0  36625.8   608.348  22319278.1389   
1  1.699547e+12  36625.8  36661.7  36571.6  36587.9   975.351  35709916.6563   
0  1.699547e+12  36587.9  36735.9  36587.9  36688.1  1035.130  37977385.1119   

            Timestamp  SMA_short_term  SMA_long_term      BB_upper  \
4 2023-11-09 16:17:00        36573.34      36894.500  37482.264403   
3 2023-11-09 16:18:00        36619.58      36873.190  37455.886479   
2 2023-11-09 16:19:00        36621.12      36847.320  37425.466478   
1 2023-11-09 16:20:00        36598.52      36816.090  37378.442101   
0 2023-11-09 16:21:00        36630.12      36788.895  37318.194121   

       BB_lower  BB_middle        RSI        MACD  Signal_line  \
4  36306.735597  36894.500  29.5

In [34]:
# オンバランスボリュームを計算して新たなカラムとして追加
df['OBV'] = calc.calculate_obv(df, 'Close', 'Volume')

# 結果を表示
print(df.tail())

      startTime     Open     High      Low    Close    Volume       turnover  \
4  1.699547e+12  36530.1  36621.3  36481.0  36545.0   995.140   36378978.228   
3  1.699547e+12  36545.0  36737.6  36506.0  36703.8   717.369  26289545.0936   
2  1.699547e+12  36703.8  36728.8  36620.0  36625.8   608.348  22319278.1389   
1  1.699547e+12  36625.8  36661.7  36571.6  36587.9   975.351  35709916.6563   
0  1.699547e+12  36587.9  36735.9  36587.9  36688.1  1035.130  37977385.1119   

            Timestamp  SMA_short_term  SMA_long_term      BB_upper  \
4 2023-11-09 16:17:00        36573.34      36894.500  37482.264403   
3 2023-11-09 16:18:00        36619.58      36873.190  37455.886479   
2 2023-11-09 16:19:00        36621.12      36847.320  37425.466478   
1 2023-11-09 16:20:00        36598.52      36816.090  37378.442101   
0 2023-11-09 16:21:00        36630.12      36788.895  37318.194121   

       BB_lower  BB_middle        RSI        MACD  Signal_line  \
4  36306.735597  36894.500  29.5

In [41]:
Timestamp=df.tail(1)['Timestamp'][0]

In [42]:
# 不要な列を削除
df = df.drop(['startTime', 'Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'turnover'], axis=1).dropna()



In [43]:
last_row = df.tail(1)
model.predict(last_row)

array([[0.23248435, 0.3829606 , 0.38455506]])

In [44]:
pre = pd.DataFrame(model.predict(last_row),columns=['pr0','pr1','pr2'])

In [45]:
df_out = pd.concat([last_row, pre], axis=1)

In [46]:
df_out.index=[Timestamp]

In [47]:
df_out.to_csv('out.csv')

In [21]:
out = pd.read_csv('out.csv')

In [23]:
df_out = pd.concat([out, df_out], axis=0)